# Test Easy Exp

In [1]:
import sys
from copy import deepcopy

In [2]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # view the path and verify

['../', '/Users/federicosiciliano/Desktop/Py_utils/easy_lightning_development/easy_exp/ntb', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/federicosiciliano/Library/Python/3.11/lib/python/site-packages', '/opt/homebrew/lib/python3.11/site-packages', '/opt/homebrew/lib/python3.11/site-packages/Easy_Lightning-0.0.1-py3.11.egg']


In [3]:
import easy_exp

## Configuration loading

In [4]:
cfg = easy_exp.cfg.load_configuration()

In [5]:
type(cfg)

easy_exp.cfg.ConfigObject

In [6]:
type(cfg["app1"]), type(cfg["__exp__"])

(dict, dict)

In [7]:
cfg["__exp__"]

{'name': 'prova',
 'project_folder': '../',
 'key_len': 16,
 'key_prefix': '',
 '__imports__': ['numpy as np',
  {'name': 'pandas', 'fromlist': ['DataFrame']},
  'multiprocessing'],
 '__wandb__': {'entity': 'sapienza_am', 'project': 'rec-posneg'},
 '__nosave__': {'app2.foo_str': None,
  'app2.foo_int2': None,
  'app2.app3.foo_int1': None,
  'app1.app_glob1_2': None},
 '__sweep__': {'parameters': {'app1.foo_sweep1': {'default': 3,
    'values': [1, 2, 3, 4, 5]},
   'app1.foo_sweep2': {'default': 3, 'min': 1, 'max': 5},
   'app1.foo_sweep3': {'default': 'a', 'values': ['a', 'b', 'c', 'd', 'e']},
   'app1.foo_sweep4': {'default': -3, 'values': [-1, -2, -3, -4, -5]},
   'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
    'min': 0.1234,
    'max': 5.6789},
   'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}},
  'method': 'random'}}

In [8]:
cfg["app1"]["foo_parse2"]

array([1, 2, 3])

In [9]:
cfg["app1.foo_parse2"]

array([1, 2, 3])

In [10]:
cfg["app3.foo_str"] = "abc"

In [11]:
cfg["app3.foo_str"],cfg["app3"]["foo_str"]

('abc', 'abc')

In [12]:
cfg["app1.foo_sweep1"]

3

In [13]:
cfg["__exp__"]["__sweep__"]

{'parameters': {'app1.foo_sweep1': {'default': 3, 'values': [1, 2, 3, 4, 5]},
  'app1.foo_sweep2': {'default': 3, 'min': 1, 'max': 5},
  'app1.foo_sweep3': {'default': 'a', 'values': ['a', 'b', 'c', 'd', 'e']},
  'app1.foo_sweep4': {'default': -3, 'values': [-1, -2, -3, -4, -5]},
  'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
   'min': 0.1234,
   'max': 5.6789},
  'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}},
 'method': 'random'}

## Experiment

In [14]:
cfg, exp_cfg = easy_exp.exp.separate_exp_cfg(cfg) # Drop __exp__ key cause it doesn't define an experiment's parameters

### Experiment ids

In [15]:
print(easy_exp.exp.generate_random_id())
print(easy_exp.exp.generate_random_id(key_prefix="B-"))
print(easy_exp.exp.generate_random_id(key_len=8))
print(easy_exp.exp.generate_random_id(key_len=4,key_prefix="FS"))

wlMyEsV8datQxZLb
B-EuljyWBOzOd8yHwl
1W54PFwF
FSJ6yQ


In [16]:
import string
import numpy as np
poss_chars = len(string.ascii_letters + string.digits) #62
default_k = 16
default_poss_keys = poss_chars ** default_k
print("Default number of possible keys: 10","^",np.log10(float(default_poss_keys)))

Default number of possible keys: 10 ^ 28.678267031972062


### Hashing

In [17]:
#The only solution, is save each numpy array / pandas DF / else in the configuration as their own experiment, having a certain experiment ID
numpy_key = "app1.foo_parse2"
np_to_save_elsewhere = cfg[numpy_key]
np_exp_id = 123 #easy_exp.exp.generate_random_id() #get numpy object experiment ID somehow
cfg[numpy_key] = np_exp_id #now cfg doesn't have the numpy array anymore

pandas_key = "app1.foo_parse3"
pd_to_save_elsewhere = cfg[pandas_key]
pd_exp_id = 42 #easy_exp.exp.generate_random_id() #get pandas object experiment ID somehow
cfg[pandas_key] = pd_exp_id #now cfg doesn't have the pandas DF anymore

In [18]:
easy_exp.exp.hash_config(cfg)

'298a9057447450466ec0fec7a66f81c4'

In [19]:
cfg

{'app1': {'foo_str': 'string_value',
  'foo_int': 123,
  'foo_float': 1.23,
  'foo_none': None,
  'foo_list1': [123, '456', 789.0],
  'foo_list2': [123, '456'],
  'foo_ref1': 'string_value',
  'foo_ref2': 'string_value',
  'foo_parse1': ['a', 'b', 'c'],
  'foo_parse2': 123,
  'foo_parse3': 42,
  'foo_parse4': 8,
  'app1_2': {'foo_str1_2': 'app2_foo', 'foo_sweep1_2': 1.2345},
  'foo_sweep1': 3,
  'foo_sweep2': 3,
  'foo_sweep3': 'a',
  'foo_sweep4': -3,
  'app_glob': 'glob',
  'app_glob2': 'glob2',
  'app_glob1_2': 'glob1_2'},
 'app2': {'app3': {'foo_int1': 123},
  'foo_str': 'app2_foo',
  'foo_int2': 123,
  'foo_sweep': 'hey'},
 'app3': {'foo_str': 'abc'}}

In [20]:
#cfg, exp_cfg = easy_exp.exp.separate_exp_cfg(cfg) # Drop __exp__ key cause it doesn't define an experiment's parameters
easy_exp.exp.get_experiment_id(cfg, exp_cfg)

(False, 'U1dnUGTKiBHugPoj')

In [21]:
cfg["app2"]

{'app3': {'foo_int1': 123},
 'foo_str': 'app2_foo',
 'foo_int2': 123,
 'foo_sweep': 'hey'}

### Save experiments

In [22]:
cfg = easy_exp.exp.combine_exp_cfg(cfg, exp_cfg)

In [23]:
cfg["__exp__"]

{'name': 'prova',
 'project_folder': '../',
 'key_len': 16,
 'key_prefix': '',
 '__imports__': ['numpy as np',
  {'name': 'pandas', 'fromlist': ['DataFrame']},
  'multiprocessing'],
 '__wandb__': {'entity': 'sapienza_am', 'project': 'rec-posneg'},
 '__nosave__': {'app2.foo_str': None,
  'app2.foo_int2': None,
  'app2.app3.foo_int1': None,
  'app1.app_glob1_2': None},
 '__sweep__': {'parameters': {'app1.foo_sweep1': {'default': 3,
    'values': [1, 2, 3, 4, 5]},
   'app1.foo_sweep2': {'default': 3, 'min': 1, 'max': 5},
   'app1.foo_sweep3': {'default': 'a', 'values': ['a', 'b', 'c', 'd', 'e']},
   'app1.foo_sweep4': {'default': -3, 'values': [-1, -2, -3, -4, -5]},
   'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
    'min': 0.1234,
    'max': 5.6789},
   'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}},
  'method': 'random'},
 'hash': '22210c3deaff792a081a2c223d7564b8'}

In [24]:
easy_exp.exp.save_experiment(cfg)
# Save experiment can be called directly; it will set the experiment id directly

In [25]:
easy_exp.exp.get_experiment_id(cfg), easy_exp.exp.get_experiment_id(cfg)
# If the cfg exists, we get
# exp_found True
# Always the same experiment_id

((True, '8yvets7OTVlBViYT'), (True, '8yvets7OTVlBViYT'))

In [26]:
cfg, exp_cfg = easy_exp.exp.separate_exp_cfg(cfg) # Drop __exp__ key cause it doesn't define an experiment's parameters

In [27]:
easy_exp.exp.get_experiment_id(cfg,exp_cfg), easy_exp.exp.get_experiment_id(cfg,exp_cfg) # Even if the cfg is divided, the result is the same

((True, '8yvets7OTVlBViYT'), (True, '8yvets7OTVlBViYT'))

In [28]:
cfg["app5"] = 5

In [29]:
easy_exp.exp.get_experiment_id(cfg,exp_cfg), easy_exp.exp.get_experiment_id(cfg,exp_cfg) # If the cfg doesn't exist, we get a different experiment_id. Also, exp_found is False

((False, 'i7GM2I0EZpTaelrH'), (False, 'WbMuFIYDaJVZOwaT'))

In [30]:
easy_exp.exp.get_set_experiment_id(cfg,exp_cfg)

(False, 'VEhDzf7aHqW0k5Nr')

In [31]:
exp_cfg["experiment_id"] # Now the experiment_id is saved inside exp_cfg

'VEhDzf7aHqW0k5Nr'

In [32]:
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]
# get_experiment_id is meant to search for the experiment in the file,
# so, if it doesn't find the cfg in the saved configurations,
# It will not set the exp_id inside exp_cfg

((False, 'ykjrahKeCEouBQyn'), 'VEhDzf7aHqW0k5Nr')

In [33]:
easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]
# In the same way, get_set_experiment_id is meant to search for the experiment in the file,
# so, if it doesn't find the cfg in the saved configurations,
# it will not return the exp_id inside exp_cfg
# and will overwrite it

((False, 'K6jtVdbjgS4ky4xq'), 'K6jtVdbjgS4ky4xq')

In [34]:
prev_cfg = deepcopy(cfg)

In [35]:
easy_exp.exp.save_experiment(cfg,exp_cfg)

In [36]:
prev_cfg == cfg # Check if cfg has been restored

True

In [37]:
# Now that the experiment is saved, we will always get the same experiment_id
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"], easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]

((True, 'K6jtVdbjgS4ky4xq'),
 'K6jtVdbjgS4ky4xq',
 (True, 'K6jtVdbjgS4ky4xq'),
 'K6jtVdbjgS4ky4xq')

In [38]:
# If we modify cfg so that is new, we get the same results as before
cfg["new_key"] = "kmasdkasm"
cfg["new_key2"] = {"abc":123}
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"], easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]

((False, 'aHsebT2HKK8xzUcs'),
 'K6jtVdbjgS4ky4xq',
 (False, 'xS7ZYmJ428pPtH5g'),
 'xS7ZYmJ428pPtH5g')

In [39]:
easy_exp.exp.save_experiment(cfg,exp_cfg)

In [40]:
exp_cfg["__nosave__"] # Check nosave keys

{'app2.foo_str': None,
 'app2.foo_int2': None,
 'app2.app3.foo_int1': None,
 'app1.app_glob1_2': None}

In [41]:
cfg["app2.foo_str"] = "modified" # Modify a nosave key

In [42]:
# If a nosave key is modified, the cfg will still match a saved one
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"], easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]

((True, 'xS7ZYmJ428pPtH5g'),
 'xS7ZYmJ428pPtH5g',
 (True, 'xS7ZYmJ428pPtH5g'),
 'xS7ZYmJ428pPtH5g')

In [43]:
cfg = easy_exp.exp.combine_exp_cfg(cfg,exp_cfg) # Combine the cfg with the exp_cfg

### Load experiments

In [44]:
all_exps = easy_exp.exp.get_experiments(**exp_cfg)
all_exps

{'v8RM2ysSFdQ348dG': {'__exp__': {'__imports__': ['numpy as np',
    {'fromlist': ['DataFrame'], 'name': 'pandas'},
    'multiprocessing'],
   '__nosave__': {'app1.app_glob1_2': None,
    'app2.app3.foo_int1': None,
    'app2.foo_int2': None,
    'app2.foo_str': None},
   '__sweep__': {'method': 'random',
    'parameters': {'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
      'max': 5.6789,
      'min': 0.1234},
     'app1.foo_sweep1': {'default': 3, 'values': [1, 2, 3, 4, 5]},
     'app1.foo_sweep2': {'default': 3, 'max': 5, 'min': 1},
     'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}}},
   '__wandb__': {'entity': 'sapienza_am', 'project': 'rec-posneg'},
   'experiment_id': 'v8RM2ysSFdQ348dG',
   'hash': '2efb2535b015311c43984c8453dd0016',
   'key_len': 16,
   'key_prefix': '',
   'name': 'prova',
   'project_folder': '../'},
  'app1': {'app1_2': {'foo_str1_2': 'app2_foo', 'foo_sweep1_2': 1.2345},
   'app_glob': 'glob',
   'app_glob2': 'glob2',
   'foo_float

In [45]:
all_exps = easy_exp.exp.get_experiments(**exp_cfg, sub_cfg={"abc":123}) # If not setting check_type, nothing will be checked
all_exps

{'v8RM2ysSFdQ348dG': {'__exp__': {'__imports__': ['numpy as np',
    {'fromlist': ['DataFrame'], 'name': 'pandas'},
    'multiprocessing'],
   '__nosave__': {'app1.app_glob1_2': None,
    'app2.app3.foo_int1': None,
    'app2.foo_int2': None,
    'app2.foo_str': None},
   '__sweep__': {'method': 'random',
    'parameters': {'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
      'max': 5.6789,
      'min': 0.1234},
     'app1.foo_sweep1': {'default': 3, 'values': [1, 2, 3, 4, 5]},
     'app1.foo_sweep2': {'default': 3, 'max': 5, 'min': 1},
     'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}}},
   '__wandb__': {'entity': 'sapienza_am', 'project': 'rec-posneg'},
   'experiment_id': 'v8RM2ysSFdQ348dG',
   'hash': '2efb2535b015311c43984c8453dd0016',
   'key_len': 16,
   'key_prefix': '',
   'name': 'prova',
   'project_folder': '../'},
  'app1': {'app1_2': {'foo_str1_2': 'app2_foo', 'foo_sweep1_2': 1.2345},
   'app_glob': 'glob',
   'app_glob2': 'glob2',
   'foo_float

In [46]:
# With contain, sub_cfg must be a "subset" of the dict
to_check = {'new_key': 'kmasdkasm','new_key2': {'abc': 123}}
all_exps = easy_exp.exp.get_experiments(**exp_cfg, sub_cfg = to_check, check_type="contain") # If not setting check_type, nothing will be checked
all_exps

{'v8RM2ysSFdQ348dG': {'__exp__': {'__imports__': ['numpy as np',
    {'fromlist': ['DataFrame'], 'name': 'pandas'},
    'multiprocessing'],
   '__nosave__': {'app1.app_glob1_2': None,
    'app2.app3.foo_int1': None,
    'app2.foo_int2': None,
    'app2.foo_str': None},
   '__sweep__': {'method': 'random',
    'parameters': {'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
      'max': 5.6789,
      'min': 0.1234},
     'app1.foo_sweep1': {'default': 3, 'values': [1, 2, 3, 4, 5]},
     'app1.foo_sweep2': {'default': 3, 'max': 5, 'min': 1},
     'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}}},
   '__wandb__': {'entity': 'sapienza_am', 'project': 'rec-posneg'},
   'experiment_id': 'v8RM2ysSFdQ348dG',
   'hash': '2efb2535b015311c43984c8453dd0016',
   'key_len': 16,
   'key_prefix': '',
   'name': 'prova',
   'project_folder': '../'},
  'app1': {'app1_2': {'foo_str1_2': 'app2_foo', 'foo_sweep1_2': 1.2345},
   'app_glob': 'glob',
   'app_glob2': 'glob2',
   'foo_float

In [47]:
#with match, sub_cfg must use relative keys
to_check = {'new_key2.abc': 123}
all_exps = easy_exp.exp.get_experiments(**exp_cfg, sub_cfg = to_check, check_type="match")
all_exps

{'v8RM2ysSFdQ348dG': {'__exp__': {'__imports__': ['numpy as np',
    {'fromlist': ['DataFrame'], 'name': 'pandas'},
    'multiprocessing'],
   '__nosave__': {'app1.app_glob1_2': None,
    'app2.app3.foo_int1': None,
    'app2.foo_int2': None,
    'app2.foo_str': None},
   '__sweep__': {'method': 'random',
    'parameters': {'app1.app1_2.foo_sweep1_2': {'default': 1.2345,
      'max': 5.6789,
      'min': 0.1234},
     'app1.foo_sweep1': {'default': 3, 'values': [1, 2, 3, 4, 5]},
     'app1.foo_sweep2': {'default': 3, 'max': 5, 'min': 1},
     'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}}},
   '__wandb__': {'entity': 'sapienza_am', 'project': 'rec-posneg'},
   'experiment_id': 'v8RM2ysSFdQ348dG',
   'hash': '2efb2535b015311c43984c8453dd0016',
   'key_len': 16,
   'key_prefix': '',
   'name': 'prova',
   'project_folder': '../'},
  'app1': {'app1_2': {'foo_str1_2': 'app2_foo', 'foo_sweep1_2': 1.2345},
   'app_glob': 'glob',
   'app_glob2': 'glob2',
   'foo_float

### Sweep parameter

In [48]:
#If one parameter is an iterable of parameters to sweep, then use sweep. It will:
#1) cycle on every value in the
#2) set the value to the key
#3) return the value
#4) At the end of the cycle, restore the iterable as value to the key
sweep_keys = ['app1.foo_sweep1','app2.foo_sweep',('app1.foo_sweep3','app1.foo_sweep4')]
print([cfg[key] if isinstance(key,str) else [cfg[sub_key] for sub_key in key] for key in sweep_keys])
for param_value in cfg.sweep(*sweep_keys):
    print(param_value,"___",[cfg[key] if isinstance(key,str) else [cfg[sub_key] for sub_key in key] for key in sweep_keys])
    easy_exp.exp.get_set_experiment_id(cfg,exp_cfg),easy_exp.exp.save_experiment(cfg,exp_cfg)
print(cfg['app1.foo_sweep1'])

[3, 'hey', ['a', -3]]
{'app1.foo_sweep1': {'default': 3, 'values': [1, 2, 3, 4, 5]}, 'app1.foo_sweep2': {'default': 3, 'min': 1, 'max': 5}, 'app1.foo_sweep3': {'default': 'a', 'values': ['a', 'b', 'c', 'd', 'e']}, 'app1.foo_sweep4': {'default': -3, 'values': [-1, -2, -3, -4, -5]}, 'app1.app1_2.foo_sweep1_2': {'default': 1.2345, 'min': 0.1234, 'max': 5.6789}, 'app2.foo_sweep': {'default': 'hey', 'values': ['hi', 'hey', 'hello']}}
REL app1.foo_sweep1
REL app2.foo_sweep
REL app1.foo_sweep3
REL app1.foo_sweep4
[1, 'hi', ('a', -1)] ___ [1, 'hi', ['a', -1]]
[1, 'hi', ('b', -2)] ___ [1, 'hi', ['b', -2]]
[1, 'hi', ('c', -3)] ___ [1, 'hi', ['c', -3]]
[1, 'hi', ('d', -4)] ___ [1, 'hi', ['d', -4]]
[1, 'hi', ('e', -5)] ___ [1, 'hi', ['e', -5]]
REL app1.foo_sweep3
REL app1.foo_sweep4
[1, 'hey', ('a', -1)] ___ [1, 'hey', ['a', -1]]
[1, 'hey', ('b', -2)] ___ [1, 'hey', ['b', -2]]
[1, 'hey', ('c', -3)] ___ [1, 'hey', ['c', -3]]
[1, 'hey', ('d', -4)] ___ [1, 'hey', ['d', -4]]
[1, 'hey', ('e', -5)] ___ 

## Sweep additions

In [ ]:
cfg = easy_exp.cfg.load_configuration("config2")

In [ ]:
cfg

{'__exp__': {'name': 'prova',
  'project_folder': '../',
  'key_len': 16,
  'key_prefix': '',
  '__nosave__': {},
  '__sweep__': {'parameters': {}}},
 'app1': {'+/app1_2': ['app1_2_file']},
 '+app2': ['app2_file', 'app2bis_file']}

In [ ]:
for value1 in cfg.sweep_additions("app2"):
    for value2 in cfg.sweep_additions("app1"):
        print(value2)
        print(cfg)
        print()

AttributeError: 'ConfigObject' object has no attribute 'sweep_additions'

In [ ]:
cfg["app2"]

In [ ]:
cfg

# Yaml loading times

In [ ]:
import yaml, os, time

In [ ]:
for i in range(10):
    with open(os.path.join("../out/exp",str(i)+".yaml"), 'w') as f:
        yaml.safe_dump(cfg["__exp__"],f)

In [ ]:
start_time = time.time()

for cont in range(10001):
    with open(os.path.join("../out/exp",str(cont%10)+".yaml"), 'r') as f:
       cfg2 = yaml.safe_load(f)

       app = cfg==cfg2

    if cont in [1,10,100,1000,10000]:
        end_time = time.time()
        print(cont,"Time/read:",(end_time-start_time)/cont)
        print(cont,"Tot time:",(end_time-start_time))
        print()

## Argparse

In [ ]:
import argparse, numpy as np

In [ ]:
parser = argparse.ArgumentParser(
                    prog = 'ProgramName',
                    description = 'What the program does',
                    epilog = 'Text at the bottom of help')

In [ ]:
parser.add_argument(
    "--foo.foo", #name or flags - Either a name or a list of option strings, e.g. foo or -f, --foo.
    #optional arguments will be identified by the - prefix; remaining arguments will be assumed to be positional
    # action - The basic type of action to be taken when this argument is encountered at the command line.
# nargs - The number of command-line arguments that should be consumed.
# const - A constant value required by some action and nargs selections.
    default = [1,2,3], #np.ones((2,3))# default - The value produced if the argument is absent from the command line and if it is absent from the namespace object.
# type - The type to which the command-line argument should be converted.
# choices - A sequence of the allowable values for the argument.
# required - Whether or not the command-line option may be omitted (optionals only).
# help - A brief description of what the argument does.
# metavar - A name for the argument in usage messages.
# dest - The name of the attribute to be added to the object returned by parse_args().
    )

In [ ]:
args = parser.parse_args(["--foo","1"])

In [ ]:
eval("np.ones((1,2))")

### Sweep

In [ ]:
cfg = easy_exp.cfg.load_configuration()

In [ ]:
easy_exp.wandb.wandb_sweep(cfg, lambda x: print(x))